# Laboratorio 3: Programación Lineal Mixta (MA4702, otoño 2023)

Miércoles 17 de Mayo.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 3 etapas. Un trabajo introductorio, una etapa presencial y otra no-presencial.
La nota global del laboratorio será 30 % (TI) + 40 % (TP) + 30 % (TNP), donde (TI) es la nota del trabajo introductorio, (TP) la nota del trabajo presencial y (TNP) la nota del trabajo no presencial.

El trabajo introductorio es de carácter estrictamente individual. La etapa presencial y no-presencial son de carácter grupal. 


# Laboratorio 3: Trabajo No-presencial (TNP)


## Instrucciones.

En la parte no presencial del laboratorio, realice las siguientes actividades:

* Complete el ejercicio 3.
* Envíe este archivo .ipynb por el módulo de tareas de UCursos en la TAREA: Laboratorio 3 completo.
* Plazo de entrega: **Sábado 20 de abril a las 18:00** (se descontará 0.5 pts. por cada hora de atraso).


<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cuál cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## A. Datos del grupo

<div class="alert alert-block alert-success">

- Número o identificador del Grupo: 
    
- Integrante 1: XXX

- Integrante 2: YYY     </div>


## B. Preparación

Desempaque el `.zip` del laboratorio en su directorio de trabajo manteniendo la estructura de sub-directorios en que vienen empaquetados los archivos. 



## Ejercicio 3

En esta etapa continuamos desarrollando la temática discutida en la Etapa Presencial y consideramos la misma notación utilizada.

Para cada una de las instancias proveídas en la Etapa Presencial del Laboratorio, se le pide resolver iterativamente $(RMP)$ y el correspondiente $(DMP)$ incorporando en cada iteración una nueva columna a $(RMP)$ de acuerdo al resultado del *Pricing Problem* y hasta que el valor óptimo del *Pricing Problem* sea no-negativo.

Para realizar lo pedido, use como base el código de la siguiente celda:

In [3]:
using DelimitedFiles, JuMP, Gurobi;           
dirInst = "dirTNP";            # Directorio con el archivo de instancias para el ejercicio 1.
const GUROBI_ENV = Gurobi.Env() 

function funcRMP(W,Paths,Pcst)

    rmp = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0, "Presolve" => 0)); 

    #
    # Agregar código a partir de aquí.
    #    
    n = size(W,1);
    N = size(Paths,1);
    @variable(rmp, x[1:N] >= 0);
    @constraint(rmp, cnstr[v in 2:n-1], sum( x[i] for i in 1:N if v in Paths[i] ) >= 1);
    @objective(rmp, Min, sum(Pcst[i]*x[i] for i in 1:N) );
    #
    # Parar de agregar código aquí.
    #
    
    return rmp, cnstr;  # cnstr[] debe ser un arreglo de referencias a las |C| restricciones (*) de (RMP).
                        # Ver: https://jump.dev/JuMP.jl/stable/manual/constraints/#jump_constraints
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-22


funcRMP (generic function with 1 method)

En la siguiente celda copie el código de la función `pricingProb(W,y,m)` que creó durante la Etapa Presencial (complete y verifique su código si no lo alcanzó a hacer durante la Etapa Presencial).

In [25]:
using Graphs
function pricingProb(W,y,m)
    # W: matriz de n x n que codifica a \widetilde{w}.
    # y: vector de dimensión n de variables duales de (RMP) como en el enunciado (con y_s = y_t = 0).
    # m: largo máximo de los (s,t)-dicaminos.
    #
    
    # Inicialización variables útiles.
    s = 1;
    t = size(W,1);    
    n = size(W,1);
    
    #
    # Agregar código a partir de aquí.
    #
    


    # Construcción de \widehat{\gamma}

    nvertices = (m+1)*n

    γ = zeros(n, n) .+ Inf

    for (index, cost) in enumerate(W)
        if cost != -1
            i = CartesianIndices(W)[index]
            γ[i] = cost - y[i[2]]
        end
    end
    
    γtilde = zeros(nvertices, nvertices) # Necesito entonces que el costo de (u, n+v) sea el mismo que (u, v)

    for node1 in nvertices
        for node2 in nvertices
            index1 = node1 % n
            index2 = node2 % n
            if index1 == 0
                index1 = n
            end
            if index2 == 0
                index2 = n
            end
            γtilde[node1, node2] = γ[index1, index2]
        end
    end

    # Cálculo de dicaminos de peso mínimo en \widehat{G} con respecto a la función de peso \widehat{\gamma}.

    # Primero construimos digrafo auxiliar G_tilde
    
    G_tilde = DiGraph(nvertices)

    for node1 in 1:nvertices
        for node2 in 1:nvertices

            level1 = node1 ÷ n
            level2 = node2 ÷ n

            if level1 == level2 - 1 # Aseguramos que el segundo nodo esté en el nivel siguiente al primero
                vertex1 = node1 % n 
                vertex2 = node2 % n 
                if vertex1 == 0 # Si el resto es 0, entonces el nodo es el n-ésimo
                    vertex1 = n
                end
                if vertex2 == 0
                    vertex2 = n
                end
                if W[vertex1, vertex2] != -1
                    add_edge!(G_tilde, node1, node2)
                end
            end
        end
    end

    # Buscamos (s,t)-dicamino costo mínimo para cada nodo t tipo k*n con k en 2:m+1 usando Bellman-Ford

    function get_path(blfs::Graphs.BellmanFordState, v::Integer) # Ver https://discourse.julialang.org/t/graph-jl-how-to-interpret-the-output-information-returned-by-the-shortest-path-algorithms/94752/2
        u = v
        path = [u]
        while blfs.parents[u] != 0
            u = blfs.parents[u]
            pushfirst!(path, u)
        end
        return path
    end
    
    ford_result = bellman_ford_shortest_paths(G_tilde, 1, γtilde)

    prices = []
    paths = []


    for k in 2:m+1
        t = k*n
        dicamn = get_path(ford_result, t)
        push!(paths, dicamn)
        push!(prices, ford_result.dists[t])
    end

    optimum_index = argmin(prices)
    optimum_price = prices[optimum_index]
    optimum_path = paths[optimum_index]


    return optimum_path, optimum_price
         # Retornar 
         #  1) tupla dicamn[] representando el (s,t)-dicamino óptimo encontrado por el pricing problem, y 
         #  2) peso prcDicamn del dicamino dicam[] con respecto a \widehat{\gamma}   
    #
    # Parar de agregar código aquí.
    #    
end

pricingProb (generic function with 1 method)

Complete el código de la siguiente celda (copie, donde se indica, el código que agregó en el Ejercicio 1b)) e inicialice (donde se índica) el arreglo con los valores de las variables duales. Finalmente, adapte (donde se indica) el código para resolver a optimalidad la relajación de $(MP)$ (hasta que el *Pricing Problem* asociado a $(RMP)$ se haga no-negativo).  

In [26]:
tmpFrm=[]; tmpRes=[]; cardC=[]; val=[]; name=[]; sts=[];  
for fn in readdir(dirInst)
    W = readdlm(joinpath(dirInst,fn), ',', Int64, '\n')
    n = size(W,1);
    s = 1;
    t = n;
    Paths = [];  Pcst = [];

    #
    # Inicialización de \mathcal{Q}: Copiar código del Ejercicio 1b)
    #
    function path_cost(path, W)
        return sum(W[path[i], path[i+1]] for i in 1:length(path)-1)
    end

    function pathtoend(vertex, n) # Aprovechando que todo nodo de C tiene una arista hacia t (spg se puede asumir agregando una arista con el costo de un v-t dicamino en G).
        t = n
        path = [1, vertex, t]
        return path
    end

    for v in 2:n-1
        path = pathtoend(v, n)
        push!(Paths, path)
        push!(Pcst, path_cost(path, W))
    end
    
    #
    # Parar de copiar código del Ejercicio 1 aquí.
    # 
    
    valPrcDicamn = -Inf;
    while valPrcDicamn < 0.0
        append!( name, [fn] );                                          # Guarda nombre del archivo con instancia
        append!( tmpFrm, @elapsed mdl,cnstr=funcRMP(W,Paths,Pcst) );    # Guarda tiempo requerido en formular (RMP)
        append!( tmpRes, @elapsed optimize!(mdl) );	                    # Guarda tiempo requerido en resolver (RMP)
        append!( cardC, size(W,1)-2 );                                  # Guarda cardinalidad de conjunto C
        if termination_status(mdl) == MOI.OPTIMAL                       # Guarda status de término y valor objetivo.
            append!( sts, ["optimo"] );
            append!( val, objective_value(mdl) );
        elseif termination_status(mdl) == MOI.INFEASIBLE
            append!( sts, ["infactible"] );
            append!( val, [Inf]);
        elseif termination_status(mdl) == MOI.DUAL_INFEASIBLE
            append!( sts, ["no_acotado"] );
            append!( val, [-Inf]);
        end       
    
  
        # Inicializar el arreglo y[] con los valores de las variables duales. Puede serle útil
        # consultar la documentación de las funciones has_duals() y dual() de JuMP has_duals().
        y = zeros(n)
        if has_duals(mdl)
            for i in 1:n
                println(cnstr)
            end
        else
            break
        end


        # Termino de inicialización de arreglo y[]
        
        dicamn, valPrcDicamn = pricingProb(W,y,3);

        # Comenzar a adaptar código aquí
        
        
        # Terminar de adaptar código aquí
    end
end

#
# NO MODIFICAR A CONTINUACION
#
# Imprime estadísticas relativas a las instancias resueltas.
println()
for i in 1:length(name)
    println( "Instancia = ", name[i] );
    println( "  |C| = ", cardC[i] );
    println( "  Tmp. formulación = ", tmpRes[i] );
    println( "  Tmp. resolución = ", tmpRes[i] );
    println( "  valor obj. = ", val[i] );
    println( "  status = ", sts[i] );
    println()
end


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:3
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:3
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.Constra

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:6
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:


    Dimension 1, 2:6
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:6
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOp

And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:6
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:6
And data, a 5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 

...

cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
 cnstr[7] : x[6] >= 1.0
 cnstr[8] : x[7] >= 1.0
 cnstr[9] : x[8] >= 1.0
 cnstr[10] : x[9] >= 1.0
 cnstr[11] : x[10] >= 1.0
 cnstr[12] : x[11] >= 1.0
 cnstr[13] : x[12] >= 1.0
 cnstr[14] : x[13] >= 1.0
 cnstr[15] : x[14] >= 1.0
 

cnstr[16] : x[15] >= 1.0
 cnstr[17] : x[16] >= 1.0
 cnstr[18] : x[17] >= 1.0
 cnstr[19] : x[18] >= 1.0
 cnstr[20] : x[19] >= 1.0
 cnstr[21] : x[20] >= 1.0
 cnstr[22] : x[21] >= 1.0
 cnstr[23] : x[22] >= 1.0
 cnstr[24] : x[23] >= 1.0
 cnstr[25] : x[24] >= 1.0
 cnstr[26] : x[25] >= 1.0
 cnstr[27] : x[26] >= 1.0
 cnstr[28] : x[27] >= 1.0
 cnstr[29] : x[28] >= 1.0
 cnstr[30] : x[29] >= 1.0


 cnstr[31] : x[30] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:31
And data, a 30-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
 cnstr[7] : x[6] >= 1.0
 cnstr[8] : x[7] >= 1.0
 cnstr[9] : x[8] >= 1.0
 cnstr[10] : x[9] >= 1.0
 cnstr[11] : x[10] >= 1.0
 cnstr[12] : x[11] >= 1.0
 cnstr[13] : x[12] >= 1.0
 cnstr[14] : x[13] >= 1.0
 cnstr[15] : x[14] >= 1.0


 cnstr[16] : x[15] >= 1.0
 cnstr[17] : x[16] >= 1.0
 cnstr[18] : x[17] >= 1.0
 cnstr[19] : x[18] >= 1.0
 cnstr[20] : x[19] >= 1.0
 cnstr[21] : x[20] >= 1.0
 cnstr[22] : x[21] >= 1.0
 cnstr[23] : x[22] >= 1.0
 cnstr[24] : x[23] >= 1.0
 cnstr[25] : x[24] >= 1.0
 cnstr[26] : x[25] >= 1.0
 cnstr[27] : x[26] >= 1.0
 cnstr[28] : x[27] >= 1.0
 cnstr[29] : x[28] >= 1.0


 cnstr[30] : x[29] >= 1.0
 cnstr[31] : x[30] >= 1.0
1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 2:31
And data, a 30-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cnstr[2] : x[1] >= 1.0
 cnstr[3] : x[2] >= 1.0
 cnstr[4] : x[3] >= 1.0
 cnstr[5] : x[4] >= 1.0
 cnstr[6] : x[5] >= 1.0
 cnstr[7] : x[6] >= 1.0
 cnstr[8] : x[7] >= 1.0
 cnstr[9] : x[8] >= 1.0
 cnstr[10] : x[9] >= 1.0
 cnstr[11] : x[10] >= 1.0
 cnstr[12] : x[11] >= 1.0
 cnstr[13] : x[12] >= 1.0
 cnstr[14] : x[13] >= 1.0
 cnstr[15] : x[14] >= 1.0
 cnstr[16] : x[15] >= 1.0
 cnstr[17] : x[16] >= 1.0
 cnstr[18] : x[17] >= 1.0
 cnstr[19] : x[18] >= 1.0
 cnstr[20] : x[19] >= 1.0
 cnstr[21] : x[20] >= 1.0
 cnstr[22] : x[21] >= 1.0
 

Observe que en algunos casos la convergencia es a un valor no integral, lo que significa que la solución óptima de la relajación de $(MP)$ encontrada no es integral -- para llegar a una solución integral habría que hacer *Branch&Price*.

<div class="alert alert-block alert-danger">Es aconsejable (pero no lo incluya en la respuesta que entregue) que genere resultados intermedios. En particular, que le permitan visualizar cómo va decreciendo el valor óptimo de $(RMP)$.</div>